# 1. Imports

In [1]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import json
import logging
import pickle
import time
import datetime

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("consumer")


config = json.load(open("config.json"))
print(json.dumps(config, indent=2))

{
  "global": {
    "kafka_bootstrap_servers": "kafka:9092",
    "kafka_topic": "test-structured-streaming",
    "kafka_consumer_group": "ss_job",
    "max_records_per_batch": 20
  }
}


# 2. Initialize Spark with Kafak Consumer

In [2]:
# Create Spark Configuration and set application name
conf = SparkConf().setAppName("KafkaExp")

# Default pyspark installation lacks kafka consumer libraries. Install kafka-client libs manually
kafka_packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{"2.12"}:{"3.3.0"}',
    'org.apache.kafka:kafka-clients:2.8.0',
    "org.apache.hadoop:hadoop-aws:3.3.0",
    "com.google.guava:guava:21.0",
    "org.apache.httpcomponents:httpcore:4.4.8"
]

# Provide kafka jar paths to driver and executors
kafka_jar_paths = '/mnt/home/prathyush/.ivy2/jars/'.join([
    "org.apache.hadoop_hadoop-client-runtime-3.3.2.jar",
    "org.apache.kafka_kafka-clients-2.8.1.jar",
    "hadoop-aws-2.7.5.jar",
    "aws-java-sdk-core-1.12.268.jar"
])

# Connect to Spark cluster (Cluster mode instead of local mode)
conf = (conf.setMaster('spark://spark:7077')
        .set('spark.jars.packages', ','.join(kafka_packages))
        .set('spark.driver.extraClassPath', '/mnt/home/prathyush/.ivy2/jars/*')
        .set('spark.executor.extraClassPath', '/mnt/home/prathyush/.ivy2/jars/*')
        )

# Create spark context
sc = SparkContext(conf=conf)

logger.info(f"Spark Driver memory: {sc._conf.get('spark.driver.memory')}")
logger.info(f"Spark Executor memory: {sc._conf.get('spark.executor.memory')}")
logger.info(
    f'Loaded jars:\n{json.dumps((sc._jsc.sc().listJars().toList().toString().replace("List(", "").replace(")", "").split(", ")), indent=2)}')
sc.setLogLevel("ERROR")

# Create spark session
spark = SparkSession(sc)
spark.conf.set("spark.sql.parquet.compression.codec", "gzip")
spark.conf.set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false")
spark.conf.set("parquet.enable.summary-metadata", "false")

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.google.guava#guava added as a dependency
org.apache.httpcomponents#httpcore added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f58a2436-6088-473b-9285-b1662c3c5ea0;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.

22/12/15 06:33:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
INFO:consumer:Spark Driver memory: None
INFO:consumer:Spark Executor memory: None
INFO:consumer:Loaded jars:
[
  "spark://41f56943b782:33451/jars/com.google.code.findbugs_jsr305-3.0.0.jar",
  "spark://41f56943b782:33451/jars/com.google.guava_guava-21.0.jar",
  "spark://41f56943b782:33451/jars/org.apache.httpcomponents_httpcore-4.4.8.jar",
  "spark://41f56943b782:33451/jars/org.apache.hadoop_hadoop-client-runtime-3.3.2.jar",
  "spark://41f56943b782:33451/jars/org.lz4_lz4-java-1.8.0.jar",
  "spark://41f56943b782:33451/jars/org.apache.commons_commons-pool2-2.11.1.jar",
  "spark://41f56943b782:33451/jars/org.apache.hadoop_hadoop-client-api-3.3.2.jar",
  "spark://41f56943b782:33451/jars/org.apache.kafka_kafka-clients-2.8.1.jar",
  "spark://41f56943b782:33451/jars/org.apache.hadoop_hadoop-aws-3.3.0.jar",
  "spark://41f56943b782:33451/jars/org.spark-project.spark_u

# 3. Test Kafka topic and connection

In [3]:
from confluent_kafka.admin import AdminClient

def test_kafka_connection(broker_conf:dict) -> None:
    """
    Function to test kafka connection
    :param broker_conf: Broker configuration
    :returns: None
    """
    client = AdminClient(broker_conf)
    topics = client.list_topics().topics
    if not topics:
        raise RuntimeError()
    print("Kafka Connection successful!")


broker_conf = {
    'bootstrap.servers': config["global"]["kafka_bootstrap_servers"]
}

# Test kafka connection
test_kafka_connection(broker_conf)


Kafka Connection successful!


# 4. Load Schema

In [4]:
# Load schema 
schema = pickle.load(open("schema.pkl", 'rb'))
print(schema)

StructType([StructField('awards', ArrayType(StructType([StructField('award', StringType(), True), StructField('by', StringType(), True), StructField('year', StringType(), True)]), True), True), StructField('birth', StringType(), True), StructField('contribs', ArrayType(StringType(), True), True), StructField('death', StringType(), True), StructField('name', StructType([StructField('aka', StringType(), True), StructField('first', StringType(), True), StructField('last', StringType(), True)]), True), StructField('title', StringType(), True)])


# 5. Configure Spark-Kafka consumer options and Subscribe to Kafka Topic

In [5]:
spark.sparkContext.setLogLevel("ERROR")

# Configure spark kafka client options
spark_kafka_options = {
    "kafka.bootstrap.servers": config["global"]["kafka_bootstrap_servers"],
    "subscribe": config["global"]["kafka_topic"],
    "kafka.group.id": config["global"]["kafka_consumer_group"],
    "maxOffsetsPerTrigger": config["global"]["max_records_per_batch"],
    "startingOffsets": "earliest",
}

# Enable spark read stream
df = spark.readStream \
.format("kafka") \
.option("kafka.bootstrap.servers", spark_kafka_options["kafka.bootstrap.servers"]) \
.option("subscribe", spark_kafka_options["subscribe"]) \
.option("kafka.group.id", spark_kafka_options["kafka.group.id"]) \
.option("maxOffsetsPerTrigger", spark_kafka_options["maxOffsetsPerTrigger"]) \
.option("startingOffsets", spark_kafka_options["startingOffsets"]) \
.load()
# .schema(schema) \

# newDF = streamDF.select(from_json(col("value"), schema).as("data"))
#    .select("data.*")
print(df)
print(type(df))

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]
<class 'pyspark.sql.dataframe.DataFrame'>


In [6]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
from pyspark.sql.functions import col, column, expr
def func_call(df, batch_id):
    df.selectExpr("CAST(value AS STRING) as json")
    requests = df.rdd.map(lambda x: x.value).collect()
    logging.info(requests)
# import spark.implicits._
# streamDF = df.selectExpr( "CAST(key AS STRING)","CAST(value AS STRING)")

streamDF = df.selectExpr("CAST(value AS STRING)").select(F.from_json(col("value"), schema).alias("data")).select("data.*")
        
# .select(F.from_json("value", schema))
streamDF.printSchema()
print(streamDF)
# new_df = df.withWatermark("timestamp", "1 seconds") \
#     .groupBy(col("value")).count()
# new_df = new_df.selectExpr("CAST(value AS STRING)", "CAST(count AS STRING)")
# query = streamDF \
#         .writeStream \
#         .format("kafka") \
#         .option("kafka.bootstrap.servers", spark_kafka_options["kafka.bootstrap.servers"]) \
#         .option("checkpointLocation", "/opt/bitnami/check7") \
#         .option("topic", "first_topic") \
#         .start()
# query.awaitTermination()
# query = streamDF \
#         .writeStream \
#         .format("console") \
#         .start()
# print(1)

root
 |-- awards: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- award: string (nullable = true)
 |    |    |-- by: string (nullable = true)
 |    |    |-- year: string (nullable = true)
 |-- birth: string (nullable = true)
 |-- contribs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- death: string (nullable = true)
 |-- name: struct (nullable = true)
 |    |-- aka: string (nullable = true)
 |    |-- first: string (nullable = true)
 |    |-- last: string (nullable = true)
 |-- title: string (nullable = true)

DataFrame[awards: array<struct<award:string,by:string,year:string>>, birth: string, contribs: array<string>, death: string, name: struct<aka:string,first:string,last:string>, title: string]


# 6. Start spark structred streaming job

In [8]:
class JsonFileHandler(object):
    def __init__(self, file_path:str, mode:str):
        """
        Initialize file handler
        """
        
        self.f = open(file_path, mode)
    
    def write_dataframe_as_jsonl(self, batch_df):
        """
        Write a micro-batch dataframe to a jsonl file 
        """
        for row in batch_df.collect():
            res = json.dumps(row.asDict())
            self.f.write(res+"\n")
    
    def close(self):
        """
        Finalize file write object
        """
        self.f.close()

In [9]:
# Lambda Function for processing each batch of record
def process_batch(batch_df, batch_idx):
    print(f"{batch_idx} | {batch_df.count()}")
    
    json_file_writer = JsonFileHandler(file_path="result.jsonl", mode="w")
    # Process data and calculate
    # a. age
    # b. num_contribs
    # c. min_max_years
    # batch_df.printSchema()
    
    batch_df = batch_df.withColumn("age",
                         F.months_between(col("death"), col("birth"))/12)
    
    from pyspark.sql.types import IntegerType
    slen = F.udf(lambda s: len(s), IntegerType())
    batch_df = batch_df.withColumn("num_contribs",
                                  slen(col("contribs")))
    batch_df = batch_df.withColumn("min_y",F.array_min(F.col('awards.year')))
    batch_df = batch_df.withColumn("max_y",F.array_max(F.col("awards.year")))
    batch_df = batch_df.withColumn("min_max", F.array(F.col("min_y"), F.col("max_y")))
    
    

    # Select required columns -  "name", "age", "num_contribs", "min_max"
    
    selectDF = batch_df.select("name", "age", "num_contribs","min_max") 

    # Save to parquet file - result.parquet 
    
    batch_df = selectDF
    json_file_writer.write_dataframe_as_jsonl(batch_df)
    json_file_writer.close()
    return batch_df

# Structred streaming query
query = streamDF \
.writeStream \
.foreachBatch(process_batch) \
.outputMode("update") \
.start()

INFO:py4j.java_gateway:Callback Server Starting
INFO:py4j.java_gateway:Socket listening on ('127.0.0.1', 39077)
INFO:py4j.clientserver:Python Server ready to receive messages
INFO:py4j.clientserver:Received command c on object id p0


0 | 10


# 7. Monitor structred streaming job progress

In [10]:
# Add startup delay
time.sleep(5)
# Update Job Status

print(query.status)
while query.status['isDataAvailable'] or query.status['isTriggerActive']:
    print(query.status)
    time.sleep(5)

# Stop query
query.stop()

logger.info("Structred streaming job completed successfully")

INFO:consumer:Structred streaming job completed successfully


{'message': 'Waiting for data to arrive', 'isDataAvailable': False, 'isTriggerActive': False}
